In [ ]:
import pandas as pd
dft=pd.read_csv('train.csv')
dft.drop(columns=['uid'],inplace=True)
dft

,day,hour,minute,C_motion,feed_water_motion,faucet_hole,vapour_pressure,vapour_enthalpy,vapour_pressure_at_division,vapour_motion,feed_water_enth,vapour_temperature,output_electricity_generation
0,Saturday,7.0,NaN,387.374315,2681.205431,0.678352,25.050893,2775.784134,26.465138,2682.758692,1289.082351,601.850322,974.182544
1,Saturday,12.0,28.0,238.153011,1604.213100,0.679706,15.241147,2903.619864,NaN,1602.318002,1144.454102,603.933579,593.915516
2,Friday,15.0,13.0,338.270228,2398.561685,0.662971,23.078376,2796.443584,24.316160,NaN,1258.098883,602.443281,884.033450
3,Saturday,14.0,NaN,254.294492,1620.034059,0.679977,15.054982,2818.023511,15.895513,1619.387667,1142.463833,588.938717,584.908719
4,Saturday,18.0,16.0,203.189410,1485.672389,0.686342,12.963420,2816.144355,13.682371,1487.685052,1099.539481,504.208816,483.609973
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50395,Saturday,3.0,53.0,358.326096,2466.188446,0.679246,23.079785,2795.566204,24.382871,2463.927286,1263.867952,NaN,907.096390
50396,Saturday,18.0,6.0,205.657731,1520.898592,0.683778,13.324238,2833.890795,14.059122,NaN,1107.134960,506.487329,497.459473
50397,Friday,NaN,4.0,356.572958,2432.402583,0.678961,22.805262,2784.953650,24.089748,NaN,1258.890934,NaN,892.699691
50398,Friday,17.0,23.0,344.533133,2447.181803,0.661900,23.493103,2803.209426,NaN,2445.952298,1264.949843,603.189838,897.466022


In [16]:
df.columns

Index(['day', 'hour', 'minute', 'C_motion', 'feed_water_motion', 'faucet_hole',
       'vapour_pressure', 'vapour_enthalpy', 'vapour_pressure_at_division',
       'vapour_motion', 'feed_water_enth', 'vapour_temperature',
       'output_electricity_generation'],
      dtype='object')

In [17]:
# Example: Soft Voting Regressor with Encoding
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

# Encode categorical columns
df = pd.get_dummies(df, columns=['day'], drop_first=True)

# Load data
X = df.drop('output_electricity_generation', axis=1)
y = df['output_electricity_generation']

# Define base models
models = [
 
    ('rf', RandomForestRegressor(n_estimators=5, random_state=42)),
    ('dt', DecisionTreeRegressor(random_state=42))
]

# Create voting ensemble
voting_reg = VotingRegressor(estimators=models)

# Cross-validate
scores = cross_val_score(voting_reg, X, y, 
                        scoring='neg_mean_squared_error', cv=5)
rmse = np.sqrt(-scores.mean())
print(f"Soft Voting RMSE: {rmse:.4f}")

Soft Voting RMSE: 4.5833


In [19]:
# Bagging Ensemble
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

X = df.drop('output_electricity_generation', axis=1)
y = df['output_electricity_generation']

# Create bagging ensemble
bagging_reg = BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    n_estimators=5,
    random_state=42
)

# Cross-validate
scores = cross_val_score(bagging_reg, X, y, 
                        scoring='neg_mean_squared_error', cv=5)
rmse = np.sqrt(-scores.mean())
print(f"Bagging RMSE: {rmse:.4f}")

Bagging RMSE: 4.2231


In [21]:
# AdaBoost Ensemble
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd

# Load data
X = df.drop('output_electricity_generation', axis=1)
y = df['output_electricity_generation']

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Create AdaBoost ensemble
adaboost_reg = AdaBoostRegressor(
    estimator=DecisionTreeRegressor(max_depth=3),  # Fix: Use `estimator` instead of `base_estimator`
    n_estimators=5,
    random_state=42
)

# Cross-validate
scores = cross_val_score(adaboost_reg, X, y, 
                        scoring='neg_mean_squared_error', cv=5)
rmse = np.sqrt(-scores.mean())
print(f"AdaBoost RMSE: {rmse:.4f}")

AdaBoost RMSE: 37.1944


In [23]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

X = df.drop('output_electricity_generation', axis=1)
y = df['output_electricity_generation']

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Create GBM model
gbm_reg = GradientBoostingRegressor(
    n_estimators=10,
    random_state=42
)

# Cross-validate
scores = cross_val_score(gbm_reg, X, y, 
                        scoring='neg_mean_squared_error', cv=5)
rmse = np.sqrt(-scores.mean())
print(f"Gradient Boosting RMSE: {rmse:.4f}")

Gradient Boosting RMSE: 66.3901


In [26]:
# XGBoost
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

X = df.drop('output_electricity_generation', axis=1)
y = df['output_electricity_generation']

# Create XGBoost model
xgb_reg = XGBRegressor(
    n_estimators=10,
    random_state=42
)

# Cross-validate
scores = cross_val_score(xgb_reg, X, y, 
                        scoring='neg_mean_squared_error', cv=5)
rmse = np.sqrt(-scores.mean())
print(f"XGBoost RMSE: {rmse:.4f}")

XGBoost RMSE: 8.7295


In [29]:
# LightGBM
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
import numpy as np

X = df.drop('output_electricity_generation', axis=1)
y = df['output_electricity_generation']

# Create LightGBM model
lgbm_reg = lgb.LGBMRegressor(
    n_estimators=10,
    random_state=42
)

# Cross-validate
scores = cross_val_score(lgbm_reg, X, y, 
                        scoring='neg_mean_squared_error', cv=5)
rmse = np.sqrt(-scores.mean())
print(f"LightGBM RMSE: {rmse:.4f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2383
[LightGBM] [Info] Number of data points in the train set: 40320, number of used features: 12
[LightGBM] [Info] Start training from score 832.067278
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000405 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2383
[LightGBM] [Info] Number of data points in the train set: 40320, number of used features: 12
[LightGBM] [Info] Start training from score 830.702511
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000351 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

In [35]:
# Stacking Ensemble
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

X = df.drop('output_electricity_generation', axis=1)
y = df['output_electricity_generation']

# Define base models
base_models = [
    ('rf', RandomForestRegressor(n_estimators=5, random_state=42)),
    ('dt', DecisionTreeRegressor(random_state=42))
]

# Create stacking ensemble
stacking_reg = StackingRegressor(
    estimators=base_models,
    final_estimator=RandomForestRegressor(n_estimators=5, random_state=42)
)

# Cross-validate
scores = cross_val_score(stacking_reg, X, y, 
                        scoring='neg_mean_squared_error', cv=5)
rmse = np.sqrt(-scores.mean())
print(f"Stacking RMSE: {rmse:.4f}")

Stacking RMSE: 4.6402
